# NEST implementation of the `astrocyte` models

#### Han-Jia Jiang, 2023-02-23

This notebook provides a reference solution for the the _astrocyte_ model in NEST.

## The problem

The equations governing the evolution of the astrocyte model are

$$\left\lbrace\begin{array}{rcl}
    \frac{d[Ca^{2+}](t)}{dt} &=& J_{leak}(t) - J_{pump}(t)  + J_{channel}(t) \\
    J_{leak}(t) &=& r_{ER,cyt} \cdot r_L \cdot \left( [Ca^{2+}]_{ER}(t) - [Ca^{2+}](t) \right) \\
    J_{channel}(t) &=& r_{ER,cyt} \cdot v_{IP_3R} \cdot m_{\infty}(t)^3 \cdot n_{\infty}(t)^3\cdot h_{IP_3R}(t)^3 \cdot \left([Ca^{2+}]_{ER} - [Ca^{2+}](t)\right) \\
    m_{\infty}(t) &=& \frac{[IP_3](t)}{[IP_3](t) + K_{IP_{3,1}}} \\
    n_{\infty}(t) &=& \frac{[Ca^{2+}](t)}{[Ca^{2+}](t) + K_{act}} \\    
    [Ca^{2+}]_{ER}(t) &=& \frac{Ca_{tot}-[Ca^{2+}](t)}{r_{ER,cyt}} \\
    \frac{dh_{IP_3}(t)}{dt} &=& \alpha_{h_{IP_3}}(t) \cdot (1-h_{IP_3}(t)) - \beta_{h_{IP_3}}(t) \cdot h_{IP_3} (t) \\
    \alpha_{h_{IP_3}}(t) &=& r_{IP_3} \cdot K_{inh} \cdot  \frac{[IP_3](t) + K_{IP_3,1}}{[IP_3](t)+K_{IP_3,2}} \\
    \beta_{h_{IP_3}}(t) &=& r_{IP_3R} \cdot [Ca^{2+}](t) \\
    \frac{d[IP_3](t)}{dt} &=& \frac{[IP_3 ]^{*} - [IP_3](t)}{\tau_{IP_3}} + r_{IP_3} \cdot \left(J^{spike}(t)\right) \\
    J^{spike}(t) &=& \sum_{s} w_{s} \cdot \delta(t-t_{s}) \\
\end{array}\right.$$


## Technical details and requirements

### Implementation of the functions

* The reference solution is implemented through [scipy](http://www.scipy.org/).

### Requirements

To run this notebook, you need:

* [numpy](http://www.numpy.org/) and [scipy](http://www.scipy.org/)


In [32]:
import numpy as np
from scipy.integrate import odeint
# import matplotlib.pyplot as plt
# %matplotlib inline
# plt.rcParams['figure.figsize'] = (15, 6)

## Reference solution
### Right hand side function

In [33]:
def rhs(y, _, p):
    '''
    Implementation of astrocyte dynamics
    
    Parameters
    ----------
    y : list
        Vector containing the state variables [IP3_astro, Ca_astro, f_IP3R_astro]
    _ : unused var
    p : Params instance
        Object containing the neuronal parameters.
        
    Returns
    -------
    dCa_astro : double
        Derivative of Ca_astro
    dIP3_astro : double
        Derivative of IP3_astro
    df_IP3R_astro : double
        Derivative of f_IP3R_astro
    '''
    IP3_astro = y[0]
    Ca_astro = y[1]
    f_IP3R_astro = y[2]

    alpha = p.r_IP3R_astro*p.K_inh_astro*(IP3_astro+p.K_IP3_1_astro)/(IP3_astro+p.K_IP3_2_astro)
    beta = p.r_IP3R_astro*Ca_astro
    Ca_ER = (p.Ca_tot_astro - Ca_astro)/p.r_ER_cyt_astro
    m_inf = IP3_astro/(IP3_astro+p.K_IP3_1_astro)
    n_inf = Ca_astro/(Ca_astro + p.K_act_astro)
    J_ch = p.r_ER_cyt_astro*p.v_IP3R_astro*((m_inf*n_inf*f_IP3R_astro)**3)*(Ca_ER - Ca_astro)
    J_pump = p.v_SERCA_astro*(Ca_astro**2)/(p.K_SERCA_astro**2 + Ca_astro**2)
    J_leak = p.r_ER_cyt_astro*p.r_L_astro*(Ca_ER - Ca_astro)

    dCa_astro = J_ch - J_pump + J_leak
    dIP3_astro = (p.IP3_0_astro - IP3_astro)/p.tau_IP3_astro
    df_IP3R_astro = alpha*(1-f_IP3R_astro) - beta*f_IP3R_astro
    
    return dIP3_astro, dCa_astro, df_IP3R_astro


### Complete model

In [34]:
def scipy_astrocyte(p, f, simtime, dt):
    '''
    Complete astrocyte model using scipy `odeint` solver.
    
    Parameters
    ----------
    p : Params instance
        Object containing the astrocyte parameters.
    f : function
        Right-hand side function
    simtime : double
        Duration of the simulation (will run between
        0 and tmax)
    dt : double
        Time increment.
        
    Returns
    -------
    t : list
        Times at which the astrocyte state was evaluated.
    y : list
        State values associated to the times in `t`
    fos : list
        List of dictionaries containing additional output
        information from `odeint`
    '''
    t = np.arange(0, simtime, dt)   # time axis
    n = len(t)                 
    y = np.zeros((n, 3))         # Ca_astro, IP3_astro, f_IP3R_astro
    y[0, 0] = 1.0
    y[0, 1] = 1.0
    y[0, 2] = 1.0
    fos = []    # full output dict from odeint()
    
    # update time-step by time-step
    for k in range(1, n):
        
        # solve ODE from t_k-1 to t_k
        d, fo = odeint(f, y[k-1, :], t[k-1:k+1], (p, ), full_output=True)
        y[k, :] = d[1, :]
        fos.append(fo)
        
    return t, y, fos

### Parameters

In [35]:
astro_param = {
    'Ca_tot_astro': 2.0,
    'IP3_0_astro': 0.16,
    'K_act_astro': 0.08234,
    'K_inh_astro': 1.049,
    'K_IP3_1_astro': 0.13,
    'K_IP3_2_astro': 0.9434,
    'K_SERCA_astro': 0.1,
    'r_ER_cyt_astro': 0.185,
    'r_IP3_astro': 5.0,
    'r_IP3R_astro': 0.0002,
    'r_L_astro': 0.00011,
    'tau_IP3_astro': 7142.0,
    'v_IP3R_astro': 0.006,
    'v_SERCA_astro': 0.0009,
}

class Params:
    '''
    Class giving access to the astrocyte
    parameters.
    '''
    def __init__(self):
        self.params = astro_param
        self.Ca_tot_astro = astro_param["Ca_tot_astro"]
        self.IP3_0_astro = astro_param["IP3_0_astro"]
        self.K_act_astro = astro_param["K_act_astro"]
        self.K_inh_astro = astro_param["K_inh_astro"]
        self.K_IP3_1_astro = astro_param["K_IP3_1_astro"]
        self.K_IP3_2_astro = astro_param["K_IP3_2_astro"]
        self.K_SERCA_astro = astro_param["K_SERCA_astro"]
        self.r_ER_cyt_astro = astro_param["r_ER_cyt_astro"]
        self.r_IP3_astro = astro_param["r_IP3_astro"]
        self.r_IP3R_astro = astro_param["r_IP3R_astro"]
        self.r_L_astro = astro_param["r_L_astro"]
        self.tau_IP3_astro = astro_param["tau_IP3_astro"]
        self.v_IP3R_astro = astro_param["v_IP3R_astro"]
        self.v_SERCA_astro = astro_param["v_SERCA_astro"]
    
p = Params()

### Simulate the implementation

In [36]:
# Parameters of the simulation
simtime = 100.
resolution = 0.01

t, y, fos = scipy_astrocyte(p, rhs, simtime, resolution)
data = np.concatenate((np.array([t]).T, y), axis=1)

np.savetxt(
    'test_astrocyte.dat', data[1:, :],
    header='Times IP3_astro Ca_astro f_IP3R_astro')


-----------------------------
### License

This file is part of NEST. Copyright (C) 2004 The NEST Initiative

NEST is free software: you can redistribute it and/or modify it under the terms of the GNU General Public License as published by the Free Software Foundation, either version 2 of the License, or (at your option) any later version.

NEST is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for more details.